In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from scipy.stats import uniform, randint, probplot, poisson, expon
import joblib

from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.neighbors import LocalOutlierFactor, KNeighborsClassifier
from sklearn.covariance import EllipticEnvelope
from sklearn.compose import make_column_transformer
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_validate
import xgboost
from sklearn.pipeline import make_pipeline
from sklearn import svm
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.figure_factory as ff
from statsmodels.tsa.seasonal import seasonal_decompose
from pandas.api.types import CategoricalDtype
from sklearn.cluster import KMeans

In [6]:
df=pd.read_csv('data_with_churn_probabilities.csv')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170589 entries, 0 to 170588
Data columns (total 28 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   client_id                                170589 non-null  int64  
 1   first_order_date                         170589 non-null  object 
 2   last_order_date                          170589 non-null  object 
 3   total_sales                              170589 non-null  float64
 4   total_quantity                           170589 non-null  int64  
 5   num_tickets                              170589 non-null  int64  
 6   num_negative_tickets                     170589 non-null  int64  
 7   sum_negative_sales                       170589 non-null  float64
 8   sum_positive_sales                       170589 non-null  float64
 9   unique_branches                          170589 non-null  int64  
 10  max_days_between_orders         

In [11]:
df.head(10)

,client_id,first_order_date,last_order_date,total_sales,total_quantity,num_tickets,num_negative_tickets,sum_negative_sales,sum_positive_sales,unique_branches,max_days_between_orders,total_period,recency,avg_days_between_orders,std_days_between_orders,monthly_frequency,avg_sale_value,refunds_per_ticket,refunds_percentage,avg_ticket_quantity,sales_total_with_respect_to_total_sales,median_days_between_orders,churn_recency_max,single_order_churn,churn_max,churn_proba,avg_monthly_sales,churn_proba_monetary
0,6,2017-10-27,2019-08-13,5.882233e+03,3038,40,0,0.0000,5.882233e+03,1,194.0,655,40,16.794872,44.097490,1.832061,147.055820,0.000000,0.000000,75.950000,6.416573e-07,0.0,0,0,0,0.004682,269.41522,1.261375
1,7,2018-07-16,2018-12-24,1.190777e+03,99,15,0,0.0000,1.190777e+03,1,161.0,161,272,11.500000,43.029060,2.795031,79.385140,0.000000,0.000000,6.600000,1.298947e-07,0.0,1,0,1,0.662992,221.88393,147.107224
2,14,2017-09-25,2019-09-20,3.043896e+06,1362031,12339,73,-51743.1450,3.095638e+06,9,10.0,725,2,0.058762,0.341219,510.579300,246.689010,0.005916,-0.016715,110.384230,3.320402e-04,0.0,0,0,0,0.008963,125954.30500,1128.983141
3,18,2017-09-26,2019-09-12,2.180872e+05,72186,2194,147,-25992.6840,2.440798e+05,2,26.0,716,10,0.326493,1.260038,91.927376,99.401620,0.067001,-0.106493,32.901550,2.378981e-05,0.0,0,0,0,0.013170,9137.72950,120.341743
4,25,2018-10-10,2018-10-10,3.745861e+02,108,4,0,0.0000,3.745861e+02,1,0.0,0,347,0.000000,0.000000,0.000000,93.646530,0.000000,0.000000,27.000000,4.086134e-08,0.0,1,0,1,0.988826,0.00000,0.000000
5,37,2017-11-14,2019-06-05,6.591136e+03,1625,59,2,-347.0700,6.938206e+03,1,216.0,568,109,9.793103,34.566364,3.116197,111.714170,0.033898,-0.050023,27.542374,7.189873e-07,0.0,0,0,0,0.115092,348.12338,40.066084
6,40,2017-09-25,2019-09-02,3.339621e+04,3348,174,1,-207.2208,3.360343e+04,5,60.0,707,20,4.086705,8.819145,7.383310,191.932220,0.005747,-0.006167,19.241379,3.642991e-06,0.0,0,0,0,0.005775,1417.09510,8.183459
7,43,2019-05-22,2019-05-22,9.955320e+01,3,1,0,0.0000,9.955320e+01,1,0.0,0,123,0.000000,0.000000,0.000000,99.553200,0.000000,0.000000,3.000000,1.085966e-08,0.0,0,0,0,0.520776,0.00000,0.000000
8,55,2018-07-03,2018-10-11,4.098118e+03,821,9,0,0.0000,4.098117e+03,1,76.0,100,346,12.500000,26.997355,2.700000,455.346400,0.000000,0.000000,91.222220,4.470389e-07,0.0,1,0,1,0.786692,1229.43530,967.186740
9,56,2017-10-26,2019-09-02,1.038817e+04,5593,177,15,-2295.3660,1.268354e+04,3,66.0,676,20,3.840909,11.254851,7.855030,58.690235,0.084746,-0.180972,31.598870,1.133183e-06,0.0,0,0,0,0.014130,461.01352,6.514210


Below is morten dataset with clv and customer segment

In [17]:
df1=pd.read_csv('clients_segments_CLV.csv')

I merge the predicted probabilities of emilio with morten's dataset

In [18]:
merged_df = pd.merge(df, df1, on='client_id', how='outer')

merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170589 entries, 0 to 170588
Data columns (total 31 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   client_id                                170589 non-null  int64  
 1   first_order_date                         170589 non-null  object 
 2   last_order_date                          170589 non-null  object 
 3   total_sales                              170589 non-null  float64
 4   total_quantity                           170589 non-null  int64  
 5   num_tickets                              170589 non-null  int64  
 6   num_negative_tickets                     170589 non-null  int64  
 7   sum_negative_sales                       170589 non-null  float64
 8   sum_positive_sales                       170589 non-null  float64
 9   unique_branches                          170589 non-null  int64  
 10  max_days_between_orders         

In [19]:
merged_df=merged_df.drop(columns='Unnamed: 0')

In [20]:
merged_df.to_csv('predicted_with_cluster.csv')

In [21]:
merged_df.head()

,client_id,first_order_date,last_order_date,total_sales,total_quantity,num_tickets,num_negative_tickets,sum_negative_sales,sum_positive_sales,unique_branches,max_days_between_orders,total_period,recency,avg_days_between_orders,std_days_between_orders,monthly_frequency,avg_sale_value,refunds_per_ticket,refunds_percentage,avg_ticket_quantity,sales_total_with_respect_to_total_sales,median_days_between_orders,churn_recency_max,single_order_churn,churn_max,churn_proba,avg_monthly_sales,churn_proba_monetary,Customer_segment,CLV
0,6,2017-10-27,2019-08-13,5.882233e+03,3038,40,0,0.000,5.882233e+03,1,194.0,655,40,16.794872,44.097490,1.832061,147.05582,0.000000,0.000000,75.95000,6.416573e-07,0.0,0,0,0,0.004682,269.41522,1.261375,Low Value Customers,1.991448e+04
1,7,2018-07-16,2018-12-24,1.190777e+03,99,15,0,0.000,1.190777e+03,1,161.0,161,272,11.500000,43.029060,2.795031,79.38514,0.000000,0.000000,6.60000,1.298947e-07,0.0,1,0,1,0.662992,221.88393,147.107224,Lost Customers,7.552510e+02
2,14,2017-09-25,2019-09-20,3.043896e+06,1362031,12339,73,-51743.145,3.095638e+06,9,10.0,725,2,0.058762,0.341219,510.579300,246.68901,0.005916,-0.016715,110.38423,3.320402e-04,0.0,0,0,0,0.008963,125954.30500,1128.983141,Top Customers,3.398026e+07
3,18,2017-09-26,2019-09-12,2.180872e+05,72186,2194,147,-25992.684,2.440798e+05,2,26.0,716,10,0.326493,1.260038,91.927376,99.40162,0.067001,-0.106493,32.90155,2.378981e-05,0.0,0,0,0,0.013170,9137.72950,120.341743,Top Customers,3.398026e+07
4,25,2018-10-10,2018-10-10,3.745861e+02,108,4,0,0.000,3.745861e+02,1,0.0,0,347,0.000000,0.000000,0.000000,93.64653,0.000000,0.000000,27.00000,4.086134e-08,0.0,1,0,1,0.988826,0.00000,0.000000,Lost Customers,7.552510e+02


In [26]:

top_10_clients_by_sales = merged_df.sort_values(by='total_sales', ascending=False).head(10)

# Display the top 10 clients by total sales
top_10_clients_by_sales

,client_id,first_order_date,last_order_date,total_sales,total_quantity,num_tickets,num_negative_tickets,sum_negative_sales,sum_positive_sales,unique_branches,max_days_between_orders,total_period,recency,avg_days_between_orders,std_days_between_orders,monthly_frequency,avg_sale_value,refunds_per_ticket,refunds_percentage,avg_ticket_quantity,sales_total_with_respect_to_total_sales,median_days_between_orders,churn_recency_max,single_order_churn,churn_max,churn_proba,avg_monthly_sales,churn_proba_monetary,Customer_segment,CLV
31226,417576,2017-09-25,2019-09-20,27040712.0,10748886,102728,47,-17530.967,27058244.0,537,4.0,725,2,0.007058,0.116319,4250.814000,263.22632,0.000458,-0.000648,104.634430,0.002950,0.0,0,0,0,0.018924,1118926.10,2.117408e+04,Top Customers,3.398026e+07
132725,1773169,2017-09-25,2019-09-20,25007702.0,47105900,71128,2170,-1318018.000,26325720.0,1,5.0,725,2,0.010193,0.140077,2943.227500,351.58730,0.030508,-0.050066,662.269400,0.002728,0.0,0,0,0,0.026785,1034801.44,2.771725e+04,Top Customers,3.398026e+07
143353,1913629,2019-04-25,2019-09-12,15569986.0,6930571,351,8,-77511.080,15647497.0,1,18.0,140,10,0.400000,2.045332,75.214290,44358.93400,0.022792,-0.004954,19745.217000,0.001698,0.0,0,0,0,0.500877,3336425.50,1.671138e+06,High value Customer,2.512519e+06
116216,1552980,2017-09-25,2019-09-20,12947281.0,13016409,56843,741,-328300.800,13275581.0,47,4.0,725,2,0.012755,0.157262,2352.124000,227.77266,0.013036,-0.024730,228.988770,0.001412,0.0,0,0,0,0.015333,535749.56,8.214900e+03,Top Customers,3.398026e+07
2408,31213,2017-09-25,2019-09-20,10101793.0,840037,17611,911,-244335.000,10346128.0,2,5.0,725,2,0.041170,0.283087,728.731000,573.60700,0.051729,-0.023616,47.699562,0.001102,0.0,0,0,0,0.020451,418005.22,8.548637e+03,Top Customers,3.398026e+07
165574,2207247,2017-09-25,2019-09-20,9730633.0,3077110,50854,548,-142117.800,9872751.0,4,4.0,725,2,0.014257,0.166200,2104.303500,191.34450,0.010776,-0.014395,60.508713,0.001061,0.0,0,0,0,0.014311,402646.88,5.762304e+03,Top Customers,3.398026e+07
24284,324012,2017-09-25,2019-09-20,9348106.0,1642289,24643,36,-4600.156,9352706.0,495,4.0,725,2,0.029421,0.237138,1019.710300,379.34125,0.001461,-0.000492,66.643230,0.001020,0.0,0,0,0,0.030574,386818.20,1.182675e+04,Top Customers,3.398026e+07
25941,346256,2017-09-25,2019-09-20,9271076.0,6018569,37795,370,-129321.414,9400398.0,7,6.0,725,2,0.019183,0.197026,1563.931000,245.29901,0.009790,-0.013757,159.242460,0.001011,0.0,0,0,0,0.015107,383630.75,5.795331e+03,Top Customers,3.398026e+07
159723,2129476,2018-09-11,2019-09-13,8725098.0,1418746,682,82,-896331.100,9621429.0,1,93.0,367,9,0.538913,5.049930,55.749317,12793.39800,0.120235,-0.093160,2080.272700,0.000952,0.0,0,0,0,0.532006,713223.25,3.794388e+05,High value Customer,2.512519e+06
68828,921974,2017-09-25,2019-09-20,8378029.5,8739573,38097,1158,-367441.750,8745472.0,17,5.0,725,2,0.019031,0.193421,1576.427600,219.91310,0.030396,-0.042015,229.403180,0.000914,0.0,0,0,0,0.015165,346677.10,5.257223e+03,Top Customers,3.398026e+07


In [25]:
top_10_urgent_client = merged_df.sort_values(by='churn_proba_monetary', ascending=False).head(10)

# Display the top 10 clients by total sales
top_10_urgent_client

,client_id,first_order_date,last_order_date,total_sales,total_quantity,num_tickets,num_negative_tickets,sum_negative_sales,sum_positive_sales,unique_branches,max_days_between_orders,total_period,recency,avg_days_between_orders,std_days_between_orders,monthly_frequency,avg_sale_value,refunds_per_ticket,refunds_percentage,avg_ticket_quantity,sales_total_with_respect_to_total_sales,median_days_between_orders,churn_recency_max,single_order_churn,churn_max,churn_proba,avg_monthly_sales,churn_proba_monetary,Customer_segment,CLV
143353,1913629,2019-04-25,2019-09-12,1.556999e+07,6930571,351,8,-77511.0800,1.564750e+07,1,18.0,140,10,0.400000,2.045332,75.21429,44358.93400,0.022792,-0.004954,19745.217000,0.001698,0.0,0,0,0,0.500877,3336425.50,1.671138e+06,High value Customer,2.512519e+06
90932,1214920,2017-12-26,2017-12-27,5.001251e+04,47,11,0,0.0000,5.001251e+04,1,1.0,1,634,0.100000,0.316228,330.00000,4546.59200,0.000000,0.000000,4.272728,0.000005,0.0,1,0,1,0.961810,1500375.20,1.443076e+06,Lost Customers,7.552510e+02
133127,1778490,2019-09-03,2019-09-04,4.188408e+04,1709,5,0,0.0000,4.188409e+04,1,1.0,1,18,0.250000,0.500000,150.00000,8376.81600,0.000000,0.000000,341.800000,0.000005,0.0,1,0,1,0.939417,1256522.50,1.180398e+06,Lost Customers,7.552510e+02
7958,106265,2018-04-23,2018-04-30,2.490679e+05,36,12,0,0.0000,2.490679e+05,1,5.0,7,510,0.636364,1.501514,51.42857,20755.66000,0.000000,0.000000,3.000000,0.000027,0.0,1,0,1,0.939706,1067433.90,1.003074e+06,Lost Customers,7.552510e+02
10598,141459,2019-02-20,2019-02-21,3.828952e+04,1581,125,3,-158.8656,3.844839e+04,1,1.0,1,213,0.008065,0.089803,3750.00000,306.31616,0.024000,-0.004132,12.648000,0.000004,0.0,1,0,1,0.863969,1148685.60,9.924293e+05,Low Value Customers,1.991448e+04
159747,2129805,2018-03-22,2018-03-23,3.614584e+04,1159,59,0,0.0000,3.614584e+04,1,1.0,1,548,0.017241,0.131306,1770.00000,612.64130,0.000000,0.000000,19.644068,0.000004,0.0,1,0,1,0.905915,1084375.10,9.823514e+05,Low Value Customers,1.991448e+04
143276,1912725,2019-08-27,2019-08-29,6.827665e+04,806,12,0,0.0000,6.827664e+04,1,2.0,2,24,0.181818,0.603023,180.00000,5689.72070,0.000000,0.000000,67.166664,0.000007,0.0,1,0,1,0.958233,1024149.75,9.813745e+05,Low Value Customers,1.991448e+04
11849,158198,2019-08-30,2019-09-02,7.867256e+04,416,4,0,0.0000,7.867256e+04,1,3.0,3,20,1.000000,1.732051,40.00000,19668.14000,0.000000,0.000000,104.000000,0.000009,0.0,1,0,1,0.972156,786725.60,7.648201e+05,Lost Customers,7.552510e+02
131809,1760771,2019-06-24,2019-06-25,2.432469e+04,241,15,0,0.0000,2.432469e+04,1,1.0,1,89,0.071429,0.267261,450.00000,1621.64590,0.000000,0.000000,16.066668,0.000003,0.0,1,0,1,0.961672,729740.60,7.017711e+05,Low Value Customers,1.991448e+04
36383,486849,2019-06-17,2019-06-18,2.073308e+04,613,5,0,0.0000,2.073308e+04,1,1.0,1,96,0.250000,0.500000,150.00000,4146.61600,0.000000,0.000000,122.600000,0.000002,0.0,1,0,1,0.987310,621992.44,6.140996e+05,Lost Customers,7.552510e+02
